# Environment set-up

### Pre-requisite
- python 3.6 or later
- pip python package manager, already available in most systems with a python interpreter installed ([pip installation instructions](https://pip.pypa.io/en/stable/installing/))

### Packages installation

Installation of the packages listed in the `requirements.txt` file, as well as the two components of the PIC-SURE API from GitHub, that is the PIC-SURE adapter and the PIC-SURE Client.
As part of this script aggregated data counts of demographics node from the source database (csv format filename - data_demo_from_db.csv ) will be compared to the aggregated counts from Picsure API dictionary for demographics node only. Separate database script is needed to extract the data in listed format. Listed is the sample data extract for Categorical and Non Categorical data nodes.

KEY,patientCount,categorical,categoryValues,observationCount,HpdsDataType,min,max
\Demographics\Age\,25000,False,,25000,phenotypes,0,90
\Demographics\Ethnicity\A-C\,9000,True,"['Abenaki Indian', 'Afghanistani', 'African', 'African American', 'Alaska Indian', 'Alaska Native', 'American', 'American Indian', 'Arab', 'Argentinean', 'Armenian', 'Asian Indian', 'Assyrian', 'Asturian', 'Attacapa Indian', 'Bahamian', 'Bangladeshi', 'Barbadian', 'Belearic Islander', 'Bhutanese', 'Bolivian', 'Brazilian', 'Burmese', 'Cambodian', 'Canadian', 'Canarian', 'Cape Verdean', 'Caribbean Islander', 'Catalonian', 'Central American', 'Central American Indian', 'Chilean', 'Chinese', 'Colombian', 'Costa Rican', 'Cuban']",10000,phenotypes,,
\Demographics\Gender\,25000,True,"['Female', 'Male', 'Unknown']",25000,phenotypes,,


In [ ]:
!cat requirements.txt

In [ ]:
import sys
!{sys.executable} -m pip install -r requirements.txt

In [ ]:
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-python-adapter-hpds.git
!{sys.executable} -m pip install --upgrade --force-reinstall git+https://github.com/hms-dbmi/pic-sure-python-client.git

Import all the external dependencies, as well as user-defined functions stored in the `python_lib` folder

In [ ]:
import json
from pprint import pprint

import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from scipy import stats

import PicSureHpdsLib
import PicSureClient


##### Setting the display parameter for tables and plots

In [ ]:
font = {'weight' : 'bold',
        'size'   : 12}
plt.rc('font', **font)

## Connecting to a PIC-SURE resource

Several information are required to get access to data through the PIC-SURE API: a network URL, a resource id, and a user-specific security token.

In [ ]:
PICSURE_network_URL = "https://precisionlink-biobank4discovery.childrens.harvard.edu/picsure/"
resource_id = "6aa47730-3288-4c45-bfa1-5a8730666016"
token_file = "token.txt"

In [ ]:
with open(token_file, "r") as f:
    my_token = f.read()

In [ ]:
client = PicSureClient.Client()
connection = client.connect(PICSURE_network_URL, my_token, True)
adapter = PicSureHpdsLib.Adapter(connection)
resource = adapter.useResource()

Two objects are created here: a `connection` and a `resource` object.

As we will only be using one single resource, **the `resource` object is actually the only one we will need to proceed with data analysis hereafter**. 

It is connected to the specific data source ID we specified, and enables to query and retrieve data from this database.

In [ ]:
demo_fullVariableDict = resource.dictionary().find("\\Demographics\\").DataFrame()
demo_fullVariableDict.to_csv('data_demo_to_csv.csv', index = True)

In [ ]:
dfapi = pd.read_csv('data_demo_to_csv.csv' )
dfapi_sort = dfapi.sort_values(by=['KEY'])
dfapi_sort

In [ ]:
dfdb = pd.read_csv('data_demo_from_db.csv' )
dfdb_sort = dfdb.sort_values(by=['KEY'])
dfdb_sort

In [ ]:
combined_dfs = pd.concat([dfapi_sort, dfdb_sort])
combined_dfs = combined_dfs.sort_values(by=['KEY'])
combined_dfs

# Below step gives the difference of the dataframes

In [ ]:
symmetric_difference = combined_dfs.drop_duplicates(keep=False)

print(symmetric_difference)
